In [ ]:
!git clone --recursive https://github.com/Microsoft/LightGBM
! cd LightGBM && rm -rf build && mkdir build && cd build && cmake -DUSE_GPU=1 ../../LightGBM && make -j4 && cd ../python-package && python3 setup.py install --precompile --gpu;

Cloning into 'LightGBM'...
remote: Enumerating objects: 100, done.
remote: Counting objects: 100% (100/100), done.
remote: Compressing objects: 100% (96/96), done.
remote: Total 20903 (delta 66), reused 13 (delta 4), pack-reused 20803
Receiving objects: 100% (20903/20903), 16.50 MiB | 30.51 MiB/s, done.
Resolving deltas: 100% (15286/15286), done.
Submodule 'include/boost/compute' (https://github.com/boostorg/compute) registered for path 'external_libs/compute'
Submodule 'eigen' (https://gitlab.com/libeigen/eigen.git) registered for path 'external_libs/eigen'
Submodule 'external_libs/fast_double_parser' (https://github.com/lemire/fast_double_parser.git) registered for path 'external_libs/fast_double_parser'
Submodule 'external_libs/fmt' (https://github.com/fmtlib/fmt.git) registered for path 'external_libs/fmt'
Cloning into '/content/LightGBM/external_libs/compute'...
remote: Enumerating objects: 21728, done.        
remote: Total 21728 (delta 0), reused 0 (delta 0), pack-reused 21728  

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas import DataFrame, concat
from datetime import date
import lightgbm as lgbm
from lightgbm import LGBMRegressor
import time
import statsmodels.api as sm
from sklearn import linear_model, metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import os
!pip install tslearn


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


     |████████████████████████████████| 778kB 4.4MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import math
from math import radians
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import MinMaxScaler
from tslearn.preprocessing import TimeSeriesScalerMinMax

In [ ]:
#### 파라미터 ####

# 하루의 틱
ticks = 48

# 예측에 사용할 일수
days = 3                # 이 부분을 바꿀 수 있다.
n_days = ticks*days

# 미래 예측할 일수
future_days = 2
future_window = ticks * future_days

### 모든변수
# ['Hour', 'Minute', 'Day', 'WS', 'Time', 'DHI','DNI','RH','T','TARGET']

# 사용할 변수
what_to_left = ['Time', 'WS', 'DHI','DNI','RH','T','TARGET', 'Day']

n_features = len(what_to_left)
n_obs = n_days * n_features # 7일 예측 기준으로 7(일 수) * 48(틱 수) * 6(변수 개수)

In [ ]:
# 데이터 불러오기 

raw_data = pd.read_csv('/content/drive/MyDrive/raw_data/train/train.csv')
submission = pd.read_csv('/content/drive/MyDrive/raw_data/sample_submission.csv')
submission.shape

(7776, 10)

In [ ]:
def drop_clms(dataset):
  dataset['Time'] = dataset['Hour'] + dataset['Minute']*(0.5/30)
  dataset['Date'] = dataset['Day']%365
  dataset.drop('Day', axis=1, inplace=True)
  
  temp = list()
  for i in range(0, len(dataset), 48):
    temp += [24-(list(dataset.DHI[i:i+48]).count(0)*0.5)]*48 

  dataset["SH"] = temp

  return dataset

def cos_time(dataset):
  dataset['sin_time'] = np.sin(2*np.pi*dataset.Time/24)
  dataset['cos_time'] = np.cos(2*np.pi*dataset.Time/24)
  return dataset

In [ ]:
def get_yday(when):
  HP = date(2020,1,1)
  results = (when-HP).days
  return results

In [ ]:
def 절기24(Date):
  target = 0
  for i in list24:
    if Date < i:
      target = list24.index(i) - 1
      break
  if Date < 5:
    target = 23
  return target

In [ ]:
def train_to_supervised(train, target, n_in):

    clmns = list(train.columns)

    # 기타 칼럼은 전과 같이 들어갈 것.
    scaled_lst = clmns

    scaled_df = train[scaled_lst]
    target_df = target

    # 미래 몇 번째 항목을 가져올 것인가
    future = [48, 96]

    ### 만약에 스케일링을 하고 싶다면 ###
    # scaled_df 데이터 프레임만 스케일링 하고, 절기랑 TARGET 데이터는 그냥 두면 된다.

    # 스케일링 해도 되고, 안해도 되는 기존에 썻던 변수들 전처리
    cols, names = list(), list()
    n_vars = 1 if type(scaled_df) is list else scaled_df.shape[1]
    n_vars2 = 1 if type(target_df) is list else target_df.shape[1]
    for i in range(n_in, 0, -1):
        cols.append(scaled_df.shift(i))
        names += [('%s(t-%d)' % (j, i)) for j in scaled_df.columns]

    # 48과 96 후의 타겟 데이터 2개 붙이기.
    # forecast sequence (t, t+1, ... t+n)
    for i in future:
        cols.append(target_df.shift(-i))
        if i == 0:
            names += [('TARGET%d(t)' % (j+1)) for j in range(n_vars2)]
        else:
            names += [('TARGET%d(t+%d)' % (j+1, i)) for j in range(n_vars2)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    agg.dropna(inplace=True)
    return agg

In [ ]:
# 머신러닝에 쓰기 위해서 재정렬 시키는 series_to_supervised 함수

def test_to_supervised(train, n_in):

    clmns = list(train.columns)

    # 타켓 칼럼의 이름을 여기에 입력
    target = ['TARGET']

    # 클래스 변수로 전환(encoding)할 칼럼을 여기에 입력
#    class_lst = ['season']

    # 기타 칼럼은 전과 같이 들어갈 것.
#    scaled_lst = list(set(clmns) - set(class_lst))

    scaled_df = train.copy()
#    class_df = train[class_lst]

    ### 만약에 스케일링을 하고 싶다면 ###
    # testset의 스케일링은 구조가 상당히 까다로우므로....... 일단 나중에 하기로 함.

    # 스케일링 해도 되고, 안해도 되는 기존에 썻던 변수들 전처리
    cols, names = list(), list()
    n_vars = 1 if type(scaled_df) is list else scaled_df.shape[1]
    for i in range(n_in, 0, -1):
        cols.append(scaled_df.shift(i))
        names += [('%s(t-%d)' % (j, i)) for j in scaled_df.columns]

    # class라 encoding 필요한 절기 끝에 하나만 붙여놓기
#    cols.append(class_df)
#    names += class_lst
    
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    agg.dropna(inplace=True)
    return agg

In [ ]:
소한 = get_yday(date(2020,1,6))
대한 = get_yday(date(2020,1,20))
입춘 = get_yday(date(2020,2,4))
우수 = get_yday(date(2020,2,19))
경칩 = get_yday(date(2020,3,6))
춘분 = get_yday(date(2020,3,21))
청명 = get_yday(date(2020,4,5))
곡우 = get_yday(date(2020,4,20))
입하 = get_yday(date(2020,5,6))
소만 = get_yday(date(2020,5,21))
망종 = get_yday(date(2020,6,6))
하지 = get_yday(date(2020,6,22))
소서 = get_yday(date(2020,7,7))
대서 = get_yday(date(2020,7,23))
입추 = get_yday(date(2020,8,8))
처서 = get_yday(date(2020,8,23))
백로 = get_yday(date(2020,9,8))
추분 = get_yday(date(2020,9,23))
한로 = get_yday(date(2020,10,8))
상강 = get_yday(date(2020,10,24))
입동 = get_yday(date(2020,11,8))
소설 = get_yday(date(2020,11,22))
대설 = get_yday(date(2020,12,7))
동지 = get_yday(date(2020,12,22))

list24 = [소한, 대한, 입춘, 우수, 경칩, 춘분, 청명, 곡우, 입하, 소만, 망종, 하지, 소서, 대서, 입추, 처서, 백로, 추분, 한로, 상강, 입동, 소설, 대설, 동지]

In [ ]:
m = 60

일출 = [7+47/m, 7+44/m, 7+34/m, 7+18/m, 6+56/m, 6+35/m, 6+12/m, 5+51/m, 5+33/m, 5+19/m, 5+11/m, 5+11/m, 5+17/m, 5+27/m, 5+41/m, 5+54/m, 6+8/m, 6+20/m, 6+33/m, 6+47/m, 7+3/m, 7+20/m, 7+33/m, 7+45/m]
일몰 = [17+28/m, 17+42/m, 17+58/m, 18+15/m, 18+30/m, 18+44/m, 18+58/m, 19+12/m, 19+25/m, 19+39/m, 19+50/m, 19+57/m, 19+56/m, 19+50/m, 19+35/m, 19+15/m, 18+51/m, 18+28/m, 18+6/m, 17+45/m, 17+28/m, 17+17/m, 17+13/m, 17+17/m]
경사각 = [32.92, 36.83, 40.75, 44.67, 48.58, 52.5, 56.42, 60.33, 64.25, 68.16, 72.01, 76, 72.1, 68.16, 64.25, 60.33, 56.42, 52.5, 48.58, 44.67, 40.75, 36.83, 32.92, 29]
남중 = [12+36/m, 12+41/m, 12+44/m, 12+44/m, 12+42/m, 12+38/m, 12+34/m, 12+29/m, 12+27/m, 12+27/m, 12+29/m, 12+32/m, 12+35/m, 12+37/m, 12+36/m, 12+33/m, 12+29/m, 12+23/m, 12+18/m, 12+15/m, 12+14/m, 12+16/m, 12+22/m, 12+29/m]
print(len(일출), len(일몰), len(경사각), len(남중))

24 24 24 24


In [ ]:
def HRA(DHI, DNI, season, hour):
  # 위도(latitude) 기준을 일단 임의로 대전으로 설정 (위도 36.19~36.2도)
  latitude = radians(36.2)
  season = int(season)
  # 절기별 대한민국의 경사각
  tilt = radians(경사각[season])

  # 절기별 대한민국 대전의 태양 남중시각
  hra = radians(15*(hour - 남중[season]))

  # 구하려는 알파
  elevation = np.arcsin(np.sin(tilt) * np.sin(latitude) + np.cos(tilt) * np.cos(latitude) * np.cos(hra))

  # 천정각(Zenith Angle)은 90 - 알파
  zenith = radians(90) - elevation

  # GHI는 DHI + DNI * cos(천정각)
  ghi = DHI + DNI *np.cos(zenith)

  return ghi

In [ ]:
def encoding(data):
  season_list = [i for i in range(0,24)]
  for k in season_list:
    data['Season_' +f'{k}'] = data['season'] == k
  return data*1

In [ ]:
def not_minus(dataset):
  for i in range(0, len(dataset.index)):
    for j in range(0, len(dataset.columns)):
      K = dataset.iloc[i,j]
      if K < 0:
        dataset.iloc[i,j] = 0
  return dataset

In [ ]:
dataset = raw_data.copy()
dataset2 = drop_clms(dataset)
dataset2.drop(['Hour', 'Minute'], axis =1, inplace=True)
dataset2

,DHI,DNI,WS,RH,T,TARGET,Time,Date,SH
0,0,0,1.5,69.08,-12,0.0,0.0,0,9.0
1,0,0,1.5,69.06,-12,0.0,0.5,0,9.0
2,0,0,1.6,71.78,-12,0.0,1.0,0,9.0
3,0,0,1.6,71.75,-12,0.0,1.5,0,9.0
4,0,0,1.6,75.20,-12,0.0,2.0,0,9.0
...,...,...,...,...,...,...,...,...,...
52555,0,0,2.4,70.70,-4,0.0,21.5,364,9.0
52556,0,0,2.4,66.79,-4,0.0,22.0,364,9.0
52557,0,0,2.2,66.78,-4,0.0,22.5,364,9.0
52558,0,0,2.1,67.72,-4,0.0,23.0,364,9.0


In [ ]:
dataset2['season'] = dataset2.apply(lambda x: 절기24(x['Date']), axis = 1)
dataset3 = cos_time(dataset2)
dataset4 = dataset3[['DHI', 'DNI', 'WS', 'RH', 'T', 'TARGET', 'SH', 'sin_time', 'cos_time','season', 'Time']]
# dataset5 = dataset4.iloc[n_days:,:-1]
# goals = pd.DataFrame(dataset4.Goal, index = dataset4.index)
dataset4

,DHI,DNI,WS,RH,T,TARGET,SH,sin_time,cos_time,season,Time
0,0,0,1.5,69.08,-12,0.0,9.0,0.000000,1.000000,23,0.0
1,0,0,1.5,69.06,-12,0.0,9.0,0.130526,0.991445,23,0.5
2,0,0,1.6,71.78,-12,0.0,9.0,0.258819,0.965926,23,1.0
3,0,0,1.6,71.75,-12,0.0,9.0,0.382683,0.923880,23,1.5
4,0,0,1.6,75.20,-12,0.0,9.0,0.500000,0.866025,23,2.0
...,...,...,...,...,...,...,...,...,...,...,...
52555,0,0,2.4,70.70,-4,0.0,9.0,-0.608761,0.793353,0,21.5
52556,0,0,2.4,66.79,-4,0.0,9.0,-0.500000,0.866025,0,22.0
52557,0,0,2.2,66.78,-4,0.0,9.0,-0.382683,0.923880,0,22.5
52558,0,0,2.1,67.72,-4,0.0,9.0,-0.258819,0.965926,0,23.0


In [ ]:
dataset5= dataset4.copy()

dataset5['GHI'] = dataset4.apply(lambda x: HRA(x.DHI, x.DNI, x.season, x.Time), axis=1)

In [ ]:
dataset5.iloc[50:51,:]

,DHI,DNI,WS,RH,T,TARGET,SH,sin_time,cos_time,season,Time,GHI
50,0,0,1.6,88.11,-11,0.0,9.0,0.258819,0.965926,23,1.0,0.0


In [ ]:
#dataset6 = dataset5[['DHI', 'DNI', 'WS', 'RH', 'T','SH', 'sin_time', 'cos_time', 'GHI', 'season', 'TARGET']]
dataset6 = dataset5[['RH', 'T', 'SH', 'sin_time', 'cos_time', 'GHI', 'TARGET']]
dataset6.head(10)

,RH,T,SH,sin_time,cos_time,GHI,TARGET
0,69.08,-12,9.0,0.000000,1.000000,0.0,0.0
1,69.06,-12,9.0,0.130526,0.991445,0.0,0.0
2,71.78,-12,9.0,0.258819,0.965926,0.0,0.0
3,71.75,-12,9.0,0.382683,0.923880,0.0,0.0
4,75.20,-12,9.0,0.500000,0.866025,0.0,0.0
5,69.29,-11,9.0,0.608761,0.793353,0.0,0.0
6,72.56,-11,9.0,0.707107,0.707107,0.0,0.0
7,72.55,-11,9.0,0.793353,0.608761,0.0,0.0
8,74.62,-11,9.0,0.866025,0.500000,0.0,0.0
9,74.61,-11,9.0,0.923880,0.382683,0.0,0.0


In [ ]:
#scaler = MinMaxScaler()
scaler = MaxAbsScaler()
#scaler = TimeSeriesScalerMinMax()
scaler.fit(dataset6)
dataset6.shape
temp_X = pd.DataFrame(scaler.transform(dataset6), columns = dataset6.columns)
temp_y = pd.DataFrame(dataset6['TARGET'])

ValueError: ignored

## dataset6 가지고 동일하게 to_supervised 함수를 써서 나누면 됩니다.
### 나머지 변수 합치기, 인코딩은 유진이가 해줄 것...

In [ ]:
dataset7 = train_to_supervised(temp_X,temp_y, n_days)
dataset7

,RH(t-144),T(t-144),SH(t-144),sin_time(t-144),cos_time(t-144),GHI(t-144),TARGET(t-144),RH(t-143),T(t-143),SH(t-143),sin_time(t-143),cos_time(t-143),GHI(t-143),TARGET(t-143),RH(t-142),T(t-142),SH(t-142),sin_time(t-142),cos_time(t-142),GHI(t-142),TARGET(t-142),RH(t-141),T(t-141),SH(t-141),sin_time(t-141),cos_time(t-141),GHI(t-141),TARGET(t-141),RH(t-140),T(t-140),SH(t-140),sin_time(t-140),cos_time(t-140),GHI(t-140),TARGET(t-140),RH(t-139),T(t-139),SH(t-139),sin_time(t-139),cos_time(t-139),...,cos_time(t-6),GHI(t-6),TARGET(t-6),RH(t-5),T(t-5),SH(t-5),sin_time(t-5),cos_time(t-5),GHI(t-5),TARGET(t-5),RH(t-4),T(t-4),SH(t-4),sin_time(t-4),cos_time(t-4),GHI(t-4),TARGET(t-4),RH(t-3),T(t-3),SH(t-3),sin_time(t-3),cos_time(t-3),GHI(t-3),TARGET(t-3),RH(t-2),T(t-2),SH(t-2),sin_time(t-2),cos_time(t-2),GHI(t-2),TARGET(t-2),RH(t-1),T(t-1),SH(t-1),sin_time(t-1),cos_time(t-1),GHI(t-1),TARGET(t-1),TARGET1(t+48),TARGET1(t+96)
144,0.6908,-0.342857,0.62069,0.000000,1.000000,0.0,0.0,0.6906,-0.342857,0.62069,0.130526,0.991445,0.0,0.0,0.7178,-0.342857,0.62069,0.258819,0.965926,0.0,0.0,0.7175,-0.342857,0.62069,0.382683,0.923880,0.0,0.0,0.7520,-0.342857,0.62069,0.500000,0.866025,0.0,0.0,0.6929,-0.314286,0.62069,0.608761,0.793353,...,0.707107,0.0,0.0,0.8289,-0.342857,0.62069,-0.608761,0.793353,0.0,0.0,0.7909,-0.371429,0.62069,-0.500000,0.866025,0.0,0.0,0.7908,-0.371429,0.62069,-0.382683,0.923880,0.0,0.0,0.8109,-0.371429,0.62069,-0.258819,0.965926,0.0,0.0,0.8108,-0.371429,0.62069,-0.130526,0.991445,0.0,0.0,0.0,0.0
145,0.6906,-0.342857,0.62069,0.130526,0.991445,0.0,0.0,0.7178,-0.342857,0.62069,0.258819,0.965926,0.0,0.0,0.7175,-0.342857,0.62069,0.382683,0.923880,0.0,0.0,0.7520,-0.342857,0.62069,0.500000,0.866025,0.0,0.0,0.6929,-0.314286,0.62069,0.608761,0.793353,0.0,0.0,0.7256,-0.314286,0.62069,0.707107,0.707107,...,0.793353,0.0,0.0,0.7909,-0.371429,0.62069,-0.500000,0.866025,0.0,0.0,0.7908,-0.371429,0.62069,-0.382683,0.923880,0.0,0.0,0.8109,-0.371429,0.62069,-0.258819,0.965926,0.0,0.0,0.8108,-0.371429,0.62069,-0.130526,0.991445,0.0,0.0,0.7776,-0.400000,0.62069,0.000000,1.000000,0.0,0.0,0.0,0.0
146,0.7178,-0.342857,0.62069,0.258819,0.965926,0.0,0.0,0.7175,-0.342857,0.62069,0.382683,0.923880,0.0,0.0,0.7520,-0.342857,0.62069,0.500000,0.866025,0.0,0.0,0.6929,-0.314286,0.62069,0.608761,0.793353,0.0,0.0,0.7256,-0.314286,0.62069,0.707107,0.707107,0.0,0.0,0.7255,-0.314286,0.62069,0.793353,0.608761,...,0.866025,0.0,0.0,0.7908,-0.371429,0.62069,-0.382683,0.923880,0.0,0.0,0.8109,-0.371429,0.62069,-0.258819,0.965926,0.0,0.0,0.8108,-0.371429,0.62069,-0.130526,0.991445,0.0,0.0,0.7776,-0.400000,0.62069,0.000000,1.000000,0.0,0.0,0.7777,-0.371429,0.62069,0.130526,0.991445,0.0,0.0,0.0,0.0
147,0.7175,-0.342857,0.62069,0.382683,0.923880,0.0,0.0,0.7520,-0.342857,0.62069,0.500000,0.866025,0.0,0.0,0.6929,-0.314286,0.62069,0.608761,0.793353,0.0,0.0,0.7256,-0.314286,0.62069,0.707107,0.707107,0.0,0.0,0.7255,-0.314286,0.62069,0.793353,0.608761,0.0,0.0,0.7462,-0.314286,0.62069,0.866025,0.500000,...,0.923880,0.0,0.0,0.8109,-0.371429,0.62069,-0.258819,0.965926,0.0,0.0,0.8108,-0.371429,0.62069,-0.130526,0.991445,0.0,0.0,0.7776,-0.400000,0.62069,0.000000,1.000000,0.0,0.0,0.7777,-0.371429,0.62069,0.130526,0.991445,0.0,0.0,0.7769,-0.371429,0.62069,0.258819,0.965926,0.0,0.0,0.0,0.0
148,0.7520,-0.342857,0.62069,0.500000,0.866025,0.0,0.0,0.6929,-0.314286,0.62069,0.608761,0.793353,0.0,0.0,0.7256,-0.314286,0.62069,0.707107,0.707107,0.0,0.0,0.7255,-0.314286,0.62069,0.793353,0.608761,0.0,0.0,0.7462,-0.314286,0.62069,0.866025,0.500000,0.0,0.0,0.7461,-0.314286,0.62069,0.923880,0.382683,...,0.965926,0.0,0.0,0.8108,-0.371429,0.62069,-0.130526,0.991445,0.0,0.0,0.7776,-0.400000,0.62069,0.000000,1.000000,0.0,0.0,0.7777,-0.371429,0.62069,0.130526,0.991445,0.0,0.0,0.7769,-0.371429,0.62069,0.258819,0.965926,0.0,0.0,0.7157,-0.371429,0.62069,0.382683,0.923880,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [ ]:
X = dataset7.iloc[:,:-2]
#X = encoding(X)
#X.drop('season', axis =1, inplace=True)

y_1 = pd.DataFrame(dataset7.iloc[:,-2])
y_2 = pd.DataFrame(dataset7.iloc[:,-1])
X

,RH(t-144),T(t-144),SH(t-144),sin_time(t-144),cos_time(t-144),GHI(t-144),TARGET(t-144),RH(t-143),T(t-143),SH(t-143),sin_time(t-143),cos_time(t-143),GHI(t-143),TARGET(t-143),RH(t-142),T(t-142),SH(t-142),sin_time(t-142),cos_time(t-142),GHI(t-142),TARGET(t-142),RH(t-141),T(t-141),SH(t-141),sin_time(t-141),cos_time(t-141),GHI(t-141),TARGET(t-141),RH(t-140),T(t-140),SH(t-140),sin_time(t-140),cos_time(t-140),GHI(t-140),TARGET(t-140),RH(t-139),T(t-139),SH(t-139),sin_time(t-139),cos_time(t-139),...,SH(t-6),sin_time(t-6),cos_time(t-6),GHI(t-6),TARGET(t-6),RH(t-5),T(t-5),SH(t-5),sin_time(t-5),cos_time(t-5),GHI(t-5),TARGET(t-5),RH(t-4),T(t-4),SH(t-4),sin_time(t-4),cos_time(t-4),GHI(t-4),TARGET(t-4),RH(t-3),T(t-3),SH(t-3),sin_time(t-3),cos_time(t-3),GHI(t-3),TARGET(t-3),RH(t-2),T(t-2),SH(t-2),sin_time(t-2),cos_time(t-2),GHI(t-2),TARGET(t-2),RH(t-1),T(t-1),SH(t-1),sin_time(t-1),cos_time(t-1),GHI(t-1),TARGET(t-1)
144,0.6908,-0.342857,0.62069,0.000000,1.000000,0.0,0.0,0.6906,-0.342857,0.62069,0.130526,0.991445,0.0,0.0,0.7178,-0.342857,0.62069,0.258819,0.965926,0.0,0.0,0.7175,-0.342857,0.62069,0.382683,0.923880,0.0,0.0,0.7520,-0.342857,0.62069,0.500000,0.866025,0.0,0.0,0.6929,-0.314286,0.62069,0.608761,0.793353,...,0.62069,-0.707107,0.707107,0.0,0.0,0.8289,-0.342857,0.62069,-0.608761,0.793353,0.0,0.0,0.7909,-0.371429,0.62069,-0.500000,0.866025,0.0,0.0,0.7908,-0.371429,0.62069,-0.382683,0.923880,0.0,0.0,0.8109,-0.371429,0.62069,-0.258819,0.965926,0.0,0.0,0.8108,-0.371429,0.62069,-0.130526,0.991445,0.0,0.0
145,0.6906,-0.342857,0.62069,0.130526,0.991445,0.0,0.0,0.7178,-0.342857,0.62069,0.258819,0.965926,0.0,0.0,0.7175,-0.342857,0.62069,0.382683,0.923880,0.0,0.0,0.7520,-0.342857,0.62069,0.500000,0.866025,0.0,0.0,0.6929,-0.314286,0.62069,0.608761,0.793353,0.0,0.0,0.7256,-0.314286,0.62069,0.707107,0.707107,...,0.62069,-0.608761,0.793353,0.0,0.0,0.7909,-0.371429,0.62069,-0.500000,0.866025,0.0,0.0,0.7908,-0.371429,0.62069,-0.382683,0.923880,0.0,0.0,0.8109,-0.371429,0.62069,-0.258819,0.965926,0.0,0.0,0.8108,-0.371429,0.62069,-0.130526,0.991445,0.0,0.0,0.7776,-0.400000,0.62069,0.000000,1.000000,0.0,0.0
146,0.7178,-0.342857,0.62069,0.258819,0.965926,0.0,0.0,0.7175,-0.342857,0.62069,0.382683,0.923880,0.0,0.0,0.7520,-0.342857,0.62069,0.500000,0.866025,0.0,0.0,0.6929,-0.314286,0.62069,0.608761,0.793353,0.0,0.0,0.7256,-0.314286,0.62069,0.707107,0.707107,0.0,0.0,0.7255,-0.314286,0.62069,0.793353,0.608761,...,0.62069,-0.500000,0.866025,0.0,0.0,0.7908,-0.371429,0.62069,-0.382683,0.923880,0.0,0.0,0.8109,-0.371429,0.62069,-0.258819,0.965926,0.0,0.0,0.8108,-0.371429,0.62069,-0.130526,0.991445,0.0,0.0,0.7776,-0.400000,0.62069,0.000000,1.000000,0.0,0.0,0.7777,-0.371429,0.62069,0.130526,0.991445,0.0,0.0
147,0.7175,-0.342857,0.62069,0.382683,0.923880,0.0,0.0,0.7520,-0.342857,0.62069,0.500000,0.866025,0.0,0.0,0.6929,-0.314286,0.62069,0.608761,0.793353,0.0,0.0,0.7256,-0.314286,0.62069,0.707107,0.707107,0.0,0.0,0.7255,-0.314286,0.62069,0.793353,0.608761,0.0,0.0,0.7462,-0.314286,0.62069,0.866025,0.500000,...,0.62069,-0.382683,0.923880,0.0,0.0,0.8109,-0.371429,0.62069,-0.258819,0.965926,0.0,0.0,0.8108,-0.371429,0.62069,-0.130526,0.991445,0.0,0.0,0.7776,-0.400000,0.62069,0.000000,1.000000,0.0,0.0,0.7777,-0.371429,0.62069,0.130526,0.991445,0.0,0.0,0.7769,-0.371429,0.62069,0.258819,0.965926,0.0,0.0
148,0.7520,-0.342857,0.62069,0.500000,0.866025,0.0,0.0,0.6929,-0.314286,0.62069,0.608761,0.793353,0.0,0.0,0.7256,-0.314286,0.62069,0.707107,0.707107,0.0,0.0,0.7255,-0.314286,0.62069,0.793353,0.608761,0.0,0.0,0.7462,-0.314286,0.62069,0.866025,0.500000,0.0,0.0,0.7461,-0.314286,0.62069,0.923880,0.382683,...,0.62069,-0.258819,0.965926,0.0,0.0,0.8108,-0.371429,0.62069,-0.130526,0.991445,0.0,0.0,0.7776,-0.400000,0.62069,0.000000,1.000000,0.0,0.0,0.7777,-0.371429,0.62069,0.130526,0.991445,0.0,0.0,0.7769,-0.371429,0.62069,0.258819,0.965926,0.0,0.0,0.7157,-0.371429,0.62069,0.382683,0.923880,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

# testset은 이미 24절기(season)가 들어있으므로, 약간 다르게 전처리를 해 줘야 함.
## (season) 구하는 과정이 생략됨.
## 이하는 testset을 구하는 코드

In [ ]:
df_test = []

# 예시 코드
# 아직 유진이 어떻게 변수 추가할 지 몰라서 to_supervised는 쓰지 않았음.

for i in range(81):
    file_path = '/content/drive/MyDrive/raw_data/test/' + str(i) + '.csv'
    new_path = '/content/drive/MyDrive/raw_data/test/adj_test/' + str(i) + '.csv'
    file_name = str(i) + '.csv'
    
    # adj_test에서 가져와야 하므로 new_path에서 불러온다
    temp = pd.read_csv(new_path)
    temp.drop('Unnamed: 0', axis = 1, inplace = True)
    testset = temp.copy()

    # 24로 저장한 24절기 season으로 이름 바꾸기
    testset.rename(columns = {'24' : 'season'}, inplace = True)

    # 이 다음은 trainset과 동일한 전처리, season은 이미 구해져 있으므로 구하지 않음
    testset2 = drop_clms(testset)
    testset2.drop(['Hour', 'Minute','Date'], axis=1,inplace=True)
    testset3 = cos_time(testset2)
    testset4 = testset3[['DHI', 'DNI', 'WS', 'RH', 'T', 'TARGET', 'SH', 'sin_time', 'cos_time','season', 'Time']]

    testset5 = testset4.copy()
    testset5['GHI'] = testset4.apply(lambda x: HRA(x.DHI, x.DNI, x.season, x.Time), axis=1)
    testset6 = testset5[['RH', 'T', 'SH', 'sin_time', 'cos_time', 'GHI', 'TARGET']] #'DHI', 'DNI', 'season', 'WS', 'SH'
    testset6 = pd.DataFrame(scaler.transform(testset6), columns=testset6.columns)
    testset7 = test_to_supervised(testset6, n_days)

#    testset8 = encoding(testset7)
#    testset8.drop('season', axis = 1, inplace = True)
    testset9 = testset7.iloc[-48:,:]

    df_test.append(testset9)

X_test = pd.concat(df_test)
# X_test = X_test.iloc[:, :n_obs]
X_test 

,RH(t-144),T(t-144),SH(t-144),sin_time(t-144),cos_time(t-144),GHI(t-144),TARGET(t-144),RH(t-143),T(t-143),SH(t-143),sin_time(t-143),cos_time(t-143),GHI(t-143),TARGET(t-143),RH(t-142),T(t-142),SH(t-142),sin_time(t-142),cos_time(t-142),GHI(t-142),TARGET(t-142),RH(t-141),T(t-141),SH(t-141),sin_time(t-141),cos_time(t-141),GHI(t-141),TARGET(t-141),RH(t-140),T(t-140),SH(t-140),sin_time(t-140),cos_time(t-140),GHI(t-140),TARGET(t-140),RH(t-139),T(t-139),SH(t-139),sin_time(t-139),cos_time(t-139),...,SH(t-6),sin_time(t-6),cos_time(t-6),GHI(t-6),TARGET(t-6),RH(t-5),T(t-5),SH(t-5),sin_time(t-5),cos_time(t-5),GHI(t-5),TARGET(t-5),RH(t-4),T(t-4),SH(t-4),sin_time(t-4),cos_time(t-4),GHI(t-4),TARGET(t-4),RH(t-3),T(t-3),SH(t-3),sin_time(t-3),cos_time(t-3),GHI(t-3),TARGET(t-3),RH(t-2),T(t-2),SH(t-2),sin_time(t-2),cos_time(t-2),GHI(t-2),TARGET(t-2),RH(t-1),T(t-1),SH(t-1),sin_time(t-1),cos_time(t-1),GHI(t-1),TARGET(t-1)
288,0.5909,0.002857,0.655172,0.000000,1.000000,0.0,0.0,0.6120,-0.011429,0.655172,0.130526,0.991445,0.0,0.0,0.6576,-0.022857,0.655172,0.258819,0.965926,0.0,0.0,0.6624,-0.025714,0.655172,0.382683,0.923880,0.0,0.0,0.6970,-0.028571,0.655172,0.500000,0.866025,0.0,0.0,0.6970,-0.028571,0.655172,0.608761,0.793353,...,0.655172,-0.707107,0.707107,0.000000,0.000000,0.8274,-0.074286,0.655172,-0.608761,0.793353,0.000000,0.000000,0.8200,-0.074286,0.655172,-0.500000,0.866025,0.000000,0.000000,0.8200,-0.074286,0.655172,-0.382683,0.923880,0.0,0.0,0.8138,-0.074286,0.655172,-0.258819,0.965926,0.0,0.0,0.8199,-0.077143,0.655172,-0.130526,0.991445,0.0,0.0
289,0.6120,-0.011429,0.655172,0.130526,0.991445,0.0,0.0,0.6576,-0.022857,0.655172,0.258819,0.965926,0.0,0.0,0.6624,-0.025714,0.655172,0.382683,0.923880,0.0,0.0,0.6970,-0.028571,0.655172,0.500000,0.866025,0.0,0.0,0.6970,-0.028571,0.655172,0.608761,0.793353,0.0,0.0,0.6817,-0.028571,0.655172,0.707107,0.707107,...,0.655172,-0.608761,0.793353,0.000000,0.000000,0.8200,-0.074286,0.655172,-0.500000,0.866025,0.000000,0.000000,0.8200,-0.074286,0.655172,-0.382683,0.923880,0.000000,0.000000,0.8138,-0.074286,0.655172,-0.258819,0.965926,0.0,0.0,0.8199,-0.077143,0.655172,-0.130526,0.991445,0.0,0.0,0.8092,-0.080000,0.655172,0.000000,1.000000,0.0,0.0
290,0.6576,-0.022857,0.655172,0.258819,0.965926,0.0,0.0,0.6624,-0.025714,0.655172,0.382683,0.923880,0.0,0.0,0.6970,-0.028571,0.655172,0.500000,0.866025,0.0,0.0,0.6970,-0.028571,0.655172,0.608761,0.793353,0.0,0.0,0.6817,-0.028571,0.655172,0.707107,0.707107,0.0,0.0,0.6768,-0.025714,0.655172,0.793353,0.608761,...,0.655172,-0.500000,0.866025,0.000000,0.000000,0.8200,-0.074286,0.655172,-0.382683,0.923880,0.000000,0.000000,0.8138,-0.074286,0.655172,-0.258819,0.965926,0.000000,0.000000,0.8199,-0.077143,0.655172,-0.130526,0.991445,0.0,0.0,0.8092,-0.080000,0.655172,0.000000,1.000000,0.0,0.0,0.8153,-0.082857,0.655172,0.130526,0.991445,0.0,0.0
291,0.6624,-0.025714,0.655172,0.382683,0.923880,0.0,0.0,0.6970,-0.028571,0.655172,0.500000,0.866025,0.0,0.0,0.6970,-0.028571,0.655172,0.608761,0.793353,0.0,0.0,0.6817,-0.028571,0.655172,0.707107,0.707107,0.0,0.0,0.6768,-0.025714,0.655172,0.793353,0.608761,0.0,0.0,0.6430,-0.020000,0.655172,0.866025,0.500000,...,0.655172,-0.382683,0.923880,0.000000,0.000000,0.8138,-0.074286,0.655172,-0.258819,0.965926,0.000000,0.000000,0.8199,-0.077143,0.655172,-0.130526,0.991445,0.000000,0.000000,0.8092,-0.080000,0.655172,0.000000,1.000000,0.0,0.0,0.8153,-0.082857,0.655172,0.130526,0.991445,0.0,0.0,0.7991,-0.085714,0.655172,0.258819,0.965926,0.0,0.0
292,0.6970,-0.028571,0.655172,0.500000,0.866025,0.0,0.0,0.6970,-0.028571,0.655172,0.608761,0.793353,0.0,0.0,0.6817,-0.028571,0.655172,0.707107,0.707107,0.0,0.0,0.6768,-0.025714,0.655172,0.793353,0.608761,0.0,0.0,0.6430,-0.020000,0.655172,0.866025,0.500000,0.0,0.0,0.6391,-0.017143,0.655172,0.923880,0.382683,...,0.655172,-0.258819,0.965926,0.000000,0.000000,0.8199,-0.077143,0.655172,-0.130526,0.991445,0.000000,0.000000,0.8092,-0.080000,0.655172,0.000000,1.000000,0.000000,0.000000,0.8153,-0.082857,0.655172,0.130526,0.99

In [ ]:
X_train_1, X_valid_1, Y_train_1,  Y_valid_1 = train_test_split(X, y_1, test_size=0.30, random_state=42)
X_train_2, X_valid_2, Y_train_2,  Y_valid_2 = train_test_split(X, y_2, test_size=0.30, random_state=42)

k=0.02
quantiles = [0.1-k, 0.2-k, 0.3-k, 0.4-k, 0.5, 0.6-k, 0.7-k, 0.8-k, 0.9-k]

In [ ]:
# LGBM 모델을 구축하고 예측까지 하는 함수를 만든다. 
#def LGBM(q, X, Y, X_test):
def LGBM(q, X_train, X_valid, Y_train, Y_valid, X_test):    
    # (a) 모델링
    model = LGBMRegressor(objective='quantile', alpha=q, learning_rate=0.007, n_estimators=15000, max_depth = 20, num_leaves = 40,\
                          min_child_samples = 10, min_child_weight = 150, min_split_gain = 0, bagging_fraction=0.7,\
                          save_binary = True, subsample=0.9, is_unbalance = False, max_drop = 50, max_bin = 63,\
                          feature_fraction = 0.7, drop_rate  = 0.1, boosting_type = "gbdt", device = 'gpu')   
         
    
        # bagging_fraction : 배깅을 하기위해서 데이터를 랜덤 샘플링하여 학습에 사용한다. 비율은 0 < fraction <= 1 이며 0이 되지 않게 해야한다.
        # learning_rate : 일반적으로 0.01 ~ 0.1 정도로 맞추고 다른 파라미터를 튜닝한다. 나중에 성능을 더 높일 때 learning rate를 더 줄인다.
        # n_estimators : 기본값은 100, 1000 정도로 해주는 게 좋고 많을 수록 과적합이 발생한 가능성이 높음.
        # save_binary = True 넣어주면 더 빨라진다고 함.
        # https://greatjoy.tistory.com/72
    
 #   splits = 3
 #   kfold = KFold(n_splits = splits, shuffle = True, random_state = 42)
 #   for n_fold, (trn_idx, val_idx) in enumerate(kfold.split(X)):
 #     X_train, Y_train = X.iloc[trn_idx], Y.iloc[trn_idx]
 #     X_valid, Y_valid = X.iloc[val_idx], Y.iloc[val_idx]
 
#    rnd_search = RandomizedSearchCV(model, param_distributions=param_dist, n_iter=100, cv=4, scoring='neg_mean_squared_error', random_state=42)
    model.fit(X_train, Y_train, eval_metric = ['quantile'], eval_set=[(X_valid, Y_valid)], early_stopping_rounds=500, verbose=500)


        # verbose : eval metric이 이 숫자만큼의 round가 지난 다음 자동으로 출력된다.
        # early_stopping_rounds : 이 숫자가 가기 전까지 validation score가 증가하지 않으면 round를 멈춘다. 

    # (b) 예측
    pred = pd.Series(model.predict(X_test).round(2))
    return pred, model

In [ ]:
# Target 예측

#def train_data(X_train, Y_train, X_test):

def train_data(X_train, Y_train, X_valid, Y_valid, X_test):


    LGBM_models=[]
    LGBM_actual_pred = pd.DataFrame()

    for q in quantiles:
        print(q)
        pred, model = LGBM(q, X_train, X_valid, Y_train, Y_valid, X_test)
        LGBM_models.append(model)
        LGBM_actual_pred = pd.concat([LGBM_actual_pred,pred],axis=1)

    LGBM_actual_pred.columns=quantiles
    
    return LGBM_models, LGBM_actual_pred

In [ ]:
tick = time.time()

# Target1
models_1, results_1 = train_data(X_train_1, Y_train_1, X_valid_1, Y_valid_1, X_test)

#models_1, results_1 = train_data(X, y_1, X_test)
results_1.sort_index()[:48]

0.08
[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=0.9 will be ignored. Current value: bagging_fraction=0.7
Training until validation scores don't improve for 500 rounds
[500]	valid_0's quantile: 1.13859
[1000]	valid_0's quantile: 1.11309
[1500]	valid_0's quantile: 1.10792
[2000]	valid_0's quantile: 1.10789
[2500]	valid_0's quantile: 1.11031
Early stopping, best iteration is:
[2175]	valid_0's quantile: 1.10677
0.18000000000000002
[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=0.9 will be ignored. Current value: bagging_fraction=0.7
Training until validation scores don't improve for 500 rounds
[500]	valid_0's quantile: 2.03639
[1000]	valid_0's quantile: 1.94426
[1500]	valid_0's quantile: 1.90282
[2000]	valid_0's quan

,0.08,0.18,0.28,0.38,0.50,0.58,0.68,0.78,0.88
0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
6,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
7,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
8,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
9,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [ ]:
# Target2
models_2, results_2 = train_data(X_train_2, Y_train_2, X_valid_2, Y_valid_2, X_test)

#models_2, results_2 = train_data(X, y_2, X_test)
tock = time.time()
results_2.sort_index()

0.08
[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=0.9 will be ignored. Current value: bagging_fraction=0.7
Training until validation scores don't improve for 500 rounds
[500]	valid_0's quantile: 1.15383
[1000]	valid_0's quantile: 1.12812
[1500]	valid_0's quantile: 1.12083
[2000]	valid_0's quantile: 1.11065
[2500]	valid_0's quantile: 1.10749
Early stopping, best iteration is:
[2290]	valid_0's quantile: 1.1065
0.18000000000000002
[LightGBM] [Warning] feature_fraction is set=0.7, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=0.9 will be ignored. Current value: bagging_fraction=0.7
Training until validation scores don't improve for 500 rounds
[500]	valid_0's quantile: 2.05825
[1000]	valid_0's quantile: 1.96224
[1500]	valid_0's quantile: 1.92578
[2000]	valid_0's quant

,0.08,0.18,0.28,0.38,0.50,0.58,0.68,0.78,0.88
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
3883,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3884,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3885,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3886,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
print('소요시간 : ', (tock - tick)/60, '분')
print(results_1.shape, results_2.shape)
#print('점수 : ', rnd_search.best_score_)
#print('최적 파라미터 : ', rnd_search.best_params_)

소요시간 :  67.90154837369919 분
(3888, 9) (3888, 9)


In [ ]:
# 출력

submission.loc[submission.id.str.contains("Day7"), "q_0.1":] = results_1.sort_index().values
submission.loc[submission.id.str.contains("Day8"), "q_0.1":] = results_2.sort_index().values
submission2 = submission.set_index('id')
submission3 = not_minus(submission2)

In [ ]:
submission3.to_csv('submission_paramtry2.csv')
!cp submission_paramtry2.csv "drive/My Drive/"